**実習ファイルをアップロードして開いたら、一番上のファイル名「クラス番号氏名」を必ず自分のものに書き換えること。**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 実習05-1 データ前処理を含むモデル作成（練習用）
---
これまでは、与えられたデータをそのまま機械学習モデルに入力しても問題なかったが、実際には、入手したデータを直接モデルに入力することは困難な場合も多く、前処理が必要な場合がある。

今回は、CSVファイルを読み込んでデータの内容を理解した後、機械学習に入力するための形式に変換する前処理を行う。
そして、最後に機械学習のモデルを作成する。

データの読み込みと前処理の手順は以下である。
1. CSVファイルの確認と読み込み
2. データの内容、性質を理解する
  *   欠損値の確認と処理
3. 入力と出力、学習用とテスト用に分ける
4. データの前処理（順番はデータの性質などによって変わるかも）
  *   列の選別
  *   欠損値の処理
  *   数値化またはダミー変数化
  *   正規化
5. データの前処理を行った後に学習や予測を行うために、pipelineを作成する。

# 問題の設定
タイタニック号の乗船者のデータから、死亡か生存かを判定する。

# 練習用データセット
行、列ともに少ないデータ（prac.csv）で練習する。

## 1. CSVファイルの確認と読み込み
### 1-1. CSVファイルの確認
データをpythonで読み込む前に、簡単に中身を確認しておく。

* 行や列のタイトル
* 行数や列数
* 中身
（各自のPCで実施する）

確認ができたら、CSVファイル（prac.csv）をGoogle Colabにアップロードする。

### 1-2. CSVファイルの読み込み
CSVファイルの処理や読み込みには、いくつかの方法があるが、pandasモジュールを使用する。
CSVファイルの読み込みには、read_csv()を使用する。

（今回は、最初の列が乗客IDであり入力には使用しないとして、index_colで行名に指定する。）
```
import pandas as pd
df = pd.read_csv('練習用ファイル名', index_col=0)
# with を使用する場合
# with open('ファイル名', 'r') as f:
#   df = pd.read_csv(f, index_col=0)
```

In [9]:
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv('prac.csv', index_col=0)

データを読み込んだ変数を表示して確認してみよう

In [10]:
# 表示して確認
df

,Survived,Pclass,Name,Sex,Age,Cabin
PassengerId,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,NaN
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,C85
3,1,3,"Heikkinen, Miss. Laina",female,26.0,NaN
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,C123
5,0,3,"Allen, Mr. William Henry",male,35.0,NaN
6,0,3,"Moran, Mr. James",male,NaN,NaN
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,E46
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,NaN
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,NaN


## 2. データの内容、性質を理解する
各データの性質（数値なのかカテゴリ変数なのか文字列なのか、など）を理解する。

> Survived：死亡or生存（目的変数、二値分類）<br>
> Pclass：旅客クラス（多値カテゴリ変数）<br>
> Name：名前（文字列）<br>
> Sex：性別（二値カテゴリ変数）<br>
> Age：年齢（数値）<br>
> Cabin：客室番号（文字列）


### 欠損値の確認
欠損値とは、何らかの理由でデータが記録されていない項目であり、
欠損値がある場合には、エラーとなる場合がある。

まずは、各列に欠損値があるか確認する。（欠損値の個数を数える）
```
#isna()：NaNならTrue、sum()：列でTrueの個数を数える
df.isna().sum()
```

In [11]:
# isna()：NaNならTrue、sum()：列でTrueの個数を数える
df.isna().sum()

,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,1
Cabin,7


今回の例では、Ageに1つ、Cabinに7つ欠損値があることが分かる。代表的な欠損値の処理としては以下がある。

*   欠損値のある**行**を削除（少ない場合）
*   欠損値のある**列**を削除（多い場合）
*   欠損値に何らかの値を入れて補完

※欠損値のある**行**を削除する場合は、入力と出力を取り出す前に実施する。
今回は、行の削除はしないことにして、次に進む。

## 3. 入力と出力、訓練用とテスト用に分ける

まず、予測したい列（Survived）を出力、それ以外の列を入力として、入力と出力に分ける。
```
X =
y =
```
その後、train_test_splitを使用して訓練用とテスト用に分ける。

**※ 練習用はデータが少ないので、すべて訓練に使用します。**

In [12]:
# 入力と出力に分ける
X = df.drop('Survived', axis=1)
y = df['Survived']

## 4. データの前処理
### 4-1. 前処理方針の決定
入力（出力以外）の各列に対して、前処理を決定する。前処理の例には、以下のようなものがある。

*   列の削除（不要、欠損値が多いなど）
*   欠損値の補完
*   数値の正規化
*   数値化またはダミー変数化（数値以外の場合など）
*   そのまま使用

入力のそれぞれの列をどう処理するか検討し、今回は以下のようにする。

### □Pclass列（客室クラス）
1等船室から3等船室まである。数値ではあるが、数量的な意味合いが無いため、ダミー変数化する。

### □Name列
名前は生死に関係ないと考えて使用しない。（削除）

### □Sex列
male、femaleはそのまま入れられないので数値化

### □Age列
欠損値が1か所なので補完する。また、値を0から1に正規化する。

### □Cabin列
欠損値が多いので使用しない。（削除）


### 4-2. 前処理の確認

#### 4-2-1. 数値化、ダミー変数化（Pclass、Sex）

データを機械学習モデルに入力するためには、数値である必要がある。また、数字であっても、数量的な意味がない場合（例：1組、2組など）は、ダミー変数化する方がよい。


*   数詞や序数詞ー＞そのまま数値にする<br />（one -> 1）
*   カテゴリ変数（二値）ー＞0,1に変換 <br />
（例：男->0、女->1）
*   カテゴリ変数(二値、多値) ー＞ダミー変数にする（one-hot encoding）<br />
（例：「赤」->[1,0,0]、青->[0,1,0]、緑->[0,0,1]）
*   順序データ ー＞数値にすることもある<br />
（例：「低」->0、「中」->1、「高」-> 2）

今回は、sklearnのモジュールを使用して以下のようにする。
> 性別を0, 1に変換にする
```
from sklearn.preprocessing import OrdinalEncoder
label = OrdinalEncoder()
label.fit_transform(df[["Sex"]]) #対応の作成と変換
```
> 客室クラスをダミー変数にする
```
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder() #カテゴリが多い場合はsparse_output=True（デフォルト）にする
onehot.fit_transform(df[["Pclass"]])#対応の作成と変換
```

In [13]:
# 性別を0, 1に変換にする
from sklearn.preprocessing import OrdinalEncoder
label = OrdinalEncoder()
label.fit_transform(df[["Sex"]])

array([[1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

In [15]:
#客室クラスをダミー変数にする
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder() #カテゴリが多い場合はsparse_output=True（デフォルト）にする
onehot.fit_transform(df[["Pclass"]])#対応の作成と変換


<10x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

元の列の表示結果と比較して、変換前後の確認をしておく。

#### 4-2-2. 値の補完と正規化（Age）
**補完：**まずは、欠損値に何か値を代入し、機械学習のモデルに入力できるようにする。様々な値が考えられるが、今回は、その他の値の平均値を使用する。
```
# Age列の欠損値を平均値で補完
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(strategy='mean')
mean_imputer.fit_transform(df[['Age']])
```


In [16]:
# Age列の欠損値を平均値で補完
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(strategy='mean')
mean_imputer.fit_transform(df[['Age']])

array([[22.        ],
       [38.        ],
       [26.        ],
       [35.        ],
       [35.        ],
       [28.11111111],
       [54.        ],
       [ 2.        ],
       [27.        ],
       [14.        ]])


**正規化：**例えば、身長：100cm～190cm、体重：30kg～90kgのように、項目によって数値に開きがある場合など、同じ範囲（0～1など）に値をそろえることが多い。

今回は、sklearnのモジュールを使用して、最小値が0、最大値が1になるように正規化する。
```
# 年齢を正規化する
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(df[["Age"]])
```

In [17]:
# 年齢を正規化する（sklearnのモジュール使用）
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(df[["Age"]])

array([[0.38461538],
       [0.69230769],
       [0.46153846],
       [0.63461538],
       [0.63461538],
       [       nan],
       [1.        ],
       [0.        ],
       [0.48076923],
       [0.23076923]])

**補完＋正規化：** 年齢の列は、補完と正規化を続けて実施する必要がある。そのため、上記の処理をまとめたパイプラインを作成する。

```
# 補完→正規化のパイプライン作成
imp_scale = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
    ])
# パイプラインを使って変換を試す
imp_scale.fit_transform(df[["Age"]])

```

In [21]:
# 補完→正規化のパイプライン作成
from sklearn.pipeline import Pipeline
imp_scale = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
    ])

# パイプラインを使って変換を試す
imp_scale.fit_transform(df[["Age"]])

array([[0.38461538],
       [0.69230769],
       [0.46153846],
       [0.63461538],
       [0.63461538],
       [0.50213675],
       [1.        ],
       [0.        ],
       [0.48076923],
       [0.23076923]])

### 1-4. 前処理を含めたモデル作成、訓練
モデルを決める。今回は二値分類であり、
例としてSVM（SVC）を使用する。可能なら他の方法でも試してみよう。
[一部参考：sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

予測を行う際は、訓練時データと全く同じ前処理を行う必要がある。特に、正規化のパラメータやダミー変数の対応は全く同じでなくてはならない。そこで、それらを含めたpipelineとしてモデルを作成する。

まず、今回、各列に実施する前処理をまとめる。

*   Name、Cabinの列：削除
*   Age：欠損値を平均値で埋めて正規化
*   Sex：数値ラベルにする
*   Pclass：ダミー変数にする

列ごとに違う前処理をするには、ColumnTransformerを使用する。

必要なモジュールをインポートする。
```
# 前処理に使用するものをインポート
from sklearn.impute import SimpleImputer  #欠損値の補完
from sklearn.preprocessing import OrdinalEncoder #数値ラベルに変換
from sklearn.preprocessing import OneHotEncoder #ダミー変数に変換
from sklearn.preprocessing import MinMaxScaler #正規化

from sklearn.compose import ColumnTransformer #列ごとに違う処理をする
from sklearn.pipeline import Pipeline # pipelineの作成

# SVM（SVC）を使用する
from sklearn.svm import SVC

```

In [22]:
# 前処理に使用するものをインポート
from sklearn.impute import SimpleImputer  #欠損値の補完
from sklearn.preprocessing import OrdinalEncoder #数値ラベルに変換
from sklearn.preprocessing import OneHotEncoder #ダミー変数に変換
from sklearn.preprocessing import MinMaxScaler #正規化

from sklearn.compose import ColumnTransformer #列ごとに違う処理をする
from sklearn.pipeline import Pipeline # pipelineの作成

# SVM（SVC）を使用する
from sklearn.svm import SVC


各列の処理を行うColumnTransformerを作成する。<br>

*   処理は、(名前, 処理, [列名のリスト]) の形で記述する。<br>**Age列はPipelineを使用する必要がある。**
*   最後に、remainderで残りの列をどうするか指定できる。デフォルトはdrop。そのまま通す場合はpassthroughを指定する。

```
# 列ごとにするときは、ColumnTransformerを使う
preproc = ColumnTransformer([
    ('drop_col','drop',["Cabin","Name"]),
    ('onehot', OneHotEncoder(handle_unknown='ignore'),["Pclass"]),
    ('enc', OrdinalEncoder(),["Sex"]),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
         ('scaler', MinMaxScaler())
         ]), ['Age']),
    ],
remainder='passthrough') # 今回は上で処理していない行は無いので指定不要
```

In [23]:
# 前処理の作成
preproc = ColumnTransformer([
    ('drop_col','drop',["Cabin","Name"]),
    ('onehot', OneHotEncoder(handle_unknown='ignore'),["Pclass"]),
    ('enc', OrdinalEncoder(),["Sex"]),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
         ('scaler', MinMaxScaler())
         ]), ['Age']),
    ],
remainder='passthrough') # 今回は上で処理していない行は無いので指定不要

この時点で、期待通りの変換が出来ているかを確認する。fit_transformを使用すると、変換ルールを作成し、変換を行う。
```
# 前処理の結果を確認
preproc.fit_transform(X)
```
変換は期待通りですか？

In [24]:
# 前処理の結果を確認
preproc.fit_transform(X)

array([[0.        , 0.        , 1.        , 1.        , 0.38461538],
       [1.        , 0.        , 0.        , 0.        , 0.69230769],
       [0.        , 0.        , 1.        , 0.        , 0.46153846],
       [1.        , 0.        , 0.        , 0.        , 0.63461538],
       [0.        , 0.        , 1.        , 1.        , 0.63461538],
       [0.        , 0.        , 1.        , 1.        , 0.50213675],
       [1.        , 0.        , 0.        , 1.        , 1.        ],
       [0.        , 0.        , 1.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.48076923],
       [0.        , 1.        , 0.        , 0.        , 0.23076923]])

前処理を行ってからモデルに入力するようにパイプラインを作成する。
```
# パイプラインの作成(前処理→モデル)
full_pipeline = Pipeline(
    [
    ('preprocess',preproc), ＃前処理
    ('model', SVC()) #モデル
    ],
    verbose=True #パイプラインの実行状況を表示
)
```

In [26]:
# パイプラインの作成
full_pipeline = Pipeline([
    ('preprocess',preproc),
    ('model', SVC()) #モデル
    ],
    verbose=True #パイプラインの実行状況を表示
)

fitを使用して学習する。練習用ではデータが少ないため、訓練用とテスト用を分けずにすべてを学習用にする。
```
# fitによる学習（自動的に視覚化してくれるはず）
full_pipeline.fit(X, y)
```

In [27]:
# fitによる学習（自動的に視覚化してくれるはず）
full_pipeline.fit(X, y)

[Pipeline] ........ (step 1 of 2) Processing preprocess, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_col', 'drop',
                                                  ['Cabin', 'Name']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Pclass']),
                                                 ('enc', OrdinalEncoder(),
                                                  ['Sex']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age'])])),
                ('model', SVC())],
         verbose=True)

scoreで精度を確認し、predictで予測ができるかを確認する。
```
# 精度の確認
full_pipeline.score(x, y)
```
```
# 予測の確認
full_pipeline.predict(x)
```

In [29]:
# 精度の確認
full_pipeline.score(X, y)

1.0

In [30]:
# 予測の確認
full_pipeline.predict(X)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

練習用は以上です。これを踏まえて、実習用のファイルを作成しましょう。

**この実習はチェック項目がないので、完成＆実行済みのファイルをダウンロードして提出してください。**